In [ ]:
!pip install transformers
!pip install yfinance
!pip install plotly

In [24]:
import requests
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import datetime
import yfinance as yf
import pandas as pd
import pytz
from plotly.graph_objs import *
import plotly.express as px
import plotly.graph_objs as go

In [25]:
API_TOKEN= 
#API_TOKEN options
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'
#API_TOKEN = 'ceep5d2ad3i92ceb1ingceep5d2ad3i92ceb1io0'

In [26]:
tokenizer = AutoTokenizer.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/distilroberta-finetuned-financial-news-sentiment-analysis")
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

def fetch_prive_n_news(stock_id, start_date, end_date):
  from datetime import datetime
  timegroup=[datetime.strptime('14:30:00', '%H:%M:%S').time(),datetime.strptime('15:30:00', '%H:%M:%S').time(),datetime.strptime('16:30:00', '%H:%M:%S').time(),datetime.strptime('17:30:00', '%H:%M:%S').time(),datetime.strptime('18:30:00', '%H:%M:%S').time(),datetime.strptime('19:30:00', '%H:%M:%S').time(),datetime.strptime('20:30:00', '%H:%M:%S').time()]
  import datetime
  raw_df =  (
        yf.Ticker(stock_id)
        .history(start=start_date, end=end_date, interval='1h')
        .tz_convert(datetime.timezone.utc)
    )
  
  raw_df['timestamp'] = raw_df.index.astype('int64') // 10**9
  output = pd.DataFrame(requests.get(f'https://finnhub.io/api/v1/company-news?symbol={stock_id}&from={start_date}&to={end_date}&token={API_TOKEN}').json())
  output['datetime'] = output['datetime'].apply(lambda x: datetime.datetime.fromtimestamp(x))
  output['sentiment'] = output['summary'].apply(lambda x: classifier(x))
  output['sentiment_group'] = output['sentiment'].apply(lambda x: x[0]['label'])
  output['sentiment_score'] = output['sentiment'].apply(lambda x: x[0]['score'])
  # datetime values in your column to the nearest half past hour.
  output['datetime_group'] = output['datetime']
  output['time_group'] = output['datetime_group'].apply(lambda x:datetime.datetime.time(x))
  output['time_group'] = output['time_group'].apply(lambda x: timegroup[0] if x<=timegroup[0] else timegroup[1] if x<=timegroup[1] else timegroup[2] if x<=timegroup[2] else timegroup[3] if x<=timegroup[3] else timegroup[4] if x<=timegroup[4] else timegroup[5] if x<=timegroup[5] else timegroup[5] if x<=timegroup[6] else x)
  output['date'] = output['datetime_group'].apply(lambda x:datetime.datetime.date(x))
  output['datetime_group']=pd.to_datetime(output.date.astype(str) + ' ' + output.time_group.astype(str))
  output['datetime_group'] = output['datetime_group'].apply(lambda x:datetime.datetime.timestamp(x))
  sentiment_results = output.groupby(['datetime_group','sentiment_group'])['category'].agg('count').reset_index()
  df_dict = dict(zip(sentiment_results['datetime_group'], sentiment_results['sentiment_group'].apply(lambda x: -1 if x == 'negative' else 1 if x == 'positive' else 0) * sentiment_results['category']))
  raw_df = raw_df.merge(pd.DataFrame.from_dict(df_dict, orient='index', columns=['sentiment_score']), left_on='timestamp', right_index=True, how='left').fillna(0)

  #Draw a price Chart and draw a second y axis for sentiment score
  #fig = go.Figure()
  #fig.add_trace(go.Scatter(x=raw_df.index, y=raw_df['Close'], name='Price'))
  #fig.add_trace(go.Bar(x=raw_df.index, y=raw_df['sentiment_score'], name='Sentiment Score', yaxis='y2', opacity=0.5))
  #fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
  #fig.show()
  return raw_df

In [ ]:
'one year news data obtaining'
import datetime as dt
end_date=dt.date.today()
result=pd.DataFrame()
for i in range(5,360,5):
    try:
      start_date= dt.date.today()- dt.timedelta(days = i)
      raw_df = fetch_prive_n_news('META', start_date,end_date)
      result=result.append(raw_df)
      end_date=dt.date.today()-dt.timedelta(days = i+1)
    except:
      result=result.append(raw_df)
      end_date=dt.date.today()-dt.timedelta(days = i+1)

 #Draw a price Chart and draw a second y axis for sentiment score
 

In [73]:
result

,Open,High,Low,Close,Volume,Dividends,Stock Splits,timestamp,sentiment_score
Datetime,,,,,,,,,
2022-12-19 14:30:00+00:00,116.830002,117.800003,115.339996,116.320000,7224894,0.0,0.0,1671460200,3.0
2022-12-19 15:30:00+00:00,116.300003,116.339996,115.650002,115.949997,4042493,0.0,0.0,1671463800,1.0
2022-12-19 16:30:00+00:00,115.959999,116.337502,115.430000,115.809998,2434663,0.0,0.0,1671467400,0.0
2022-12-19 17:30:00+00:00,115.805000,116.790001,115.805000,115.900002,3177323,0.0,0.0,1671471000,0.0
2022-12-19 18:30:00+00:00,115.880096,115.900002,114.629997,114.739998,2804227,0.0,0.0,1671474600,2.0
...,...,...,...,...,...,...,...,...,...
2022-01-05 16:30:00+00:00,334.250000,334.570007,331.204590,332.230011,1506512,0.0,0.0,1641400200,0.0
2022-01-05 17:30:00+00:00,332.239990,332.239990,328.410004,329.765015,1856220,0.0,0.0,1641403800,0.0
2022-01-05 18:30:00+00:00,329.686615,331.989990,328.019989,328.494995,1675691,0.0,0.0,1641407400,-1.0


In [74]:
result1=pd.DataFrame()
result1=result.sort_index(ascending=True)
result1

,Open,High,Low,Close,Volume,Dividends,Stock Splits,timestamp,sentiment_score
Datetime,,,,,,,,,
2022-01-03 14:30:00+00:00,338.295013,341.081604,337.190002,340.290009,3806255,0.0,0.0,1641220200,0.0
2022-01-03 15:30:00+00:00,340.290009,340.625000,338.239990,338.619995,1686041,0.0,0.0,1641223800,0.0
2022-01-03 16:30:00+00:00,338.510010,339.089996,337.660004,338.220001,984690,0.0,0.0,1641227400,-1.0
2022-01-03 17:30:00+00:00,338.274994,339.899994,337.899994,339.000000,986709,0.0,0.0,1641231000,0.0
2022-01-03 18:30:00+00:00,339.079987,339.359985,338.500000,338.940002,855723,0.0,0.0,1641234600,0.0
...,...,...,...,...,...,...,...,...,...
2022-12-22 16:30:00+00:00,116.330002,116.519997,115.150002,115.250000,3218664,0.0,0.0,1671726600,0.0
2022-12-22 17:30:00+00:00,115.230003,115.500000,114.379997,114.599998,1879433,0.0,0.0,1671730200,0.0
2022-12-22 18:30:00+00:00,114.610001,115.699997,114.550003,115.599998,2249523,0.0,0.0,1671733800,0.0


In [78]:
 fig = go.Figure()
 fig.add_trace(go.Scatter(x=result1.index, y=result1['Close'], name='Price',marker={'color':'blue','symbol':'circle'}))
 fig.add_trace(go.Bar(x=result1.index, y=result1['sentiment_score'], name='Sentiment Score', yaxis='y2', opacity=0.8, marker={'color':'red'}))
 fig.update_layout(yaxis2=dict(overlaying='y', side='right'))
 fig.show()